# Practica 9: Conjuntos de árboles con XGBoost

Grupo 5: Jorge Ortega y Daniela Vidal

In [74]:
RANDOM_STATE=83

## Conjunto de datos

En este notebook vamos a usar el conjunto de datos sobre flores del Iris.

In [75]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

# Cargamos el dataset del iris
iris = load_iris()
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [76]:
iris['feature_names']

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [77]:
import pandas as pd

df = pd.DataFrame(data=iris['data'], columns=iris['feature_names']) 
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [78]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
sepal length (cm),150.0,5.843333,0.828066,4.3,5.1,5.80,6.4,7.9
sepal width (cm),150.0,3.057333,0.435866,2.0,2.8,3.00,3.3,4.4
petal length (cm),150.0,3.758000,1.765298,1.0,1.6,4.35,5.1,6.9
petal width (cm),150.0,1.199333,0.762238,0.1,0.3,1.30,1.8,2.5


Separamos el conjunto de datos en entrenamiento, test y validación.

In [84]:
from sklearn.model_selection import train_test_split

X_train, X, y_train, y = train_test_split(iris['data'], iris['target'], test_size=0.4, random_state=RANDOM_STATE)
X_val, X_test, y_val, y_test = train_test_split(X, y, test_size=0.5, random_state=RANDOM_STATE)

## Entrenamos el árbol con XGBoost

Vamos a construir un árbol de decisión usando los parámetros por defecto.

In [80]:
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier

Vemos el arbol sin estudiar los parámetros.

In [85]:

model = XGBClassifier(objective='binary:logistic', n_estimators=10, seed=RANDOM_STATE)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
y_pred = model.predict(X_test)

In [86]:
accuracy = (np.sum(y_pred == y_test) / y_test.shape[0]) * 100
print("Accuracy: %.2f%%" % accuracy)

Accuracy: 90.00%


In [87]:
acurracy_score = model.score(X_test, y_test)
print("Accuracy score: %.2f%%" % (acurracy_score * 100.0))

Accuracy score: 90.00%


In [88]:
val_accuracy = model.score(X_val, y_val)
print("Val accuracy: %.2f%%" % (val_accuracy * 100.0))

Val accuracy: 96.67%


Los parámetros más importantes son:

objective: en el que indicamos que vamos a hacer un problema de clasificación binaria.

n_estimators: el número de árboles que vamos a usar.

max_depth: la profundidad máxima de cada árbol.

learning_rate: la tasa de aprendizaje.

Veamos cuales son los mejores parámetros para este problema.

In [89]:
n_estimators = [10, 50, 100, 200, 300]
max_depths = [2, 3, 4, 7, 10]
learning_rates = [0.001, 0.01, 0.1, 0.2, 0.3]

In [90]:
model = XGBClassifier(objective='binary:logistic', seed=RANDOM_STATE)

for ne in n_estimators:
    for md in max_depths:
        for lr in learning_rates:
            model.set_params(n_estimators=ne, max_depth=md, learning_rate=lr)
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

            train_accuracy = model.score(X_train, y_train) *100
            test_accuracy = model.score(X_test, y_test) *100
            
            print("Train accuracy: %.2f%%" % train_accuracy, "Test accuracy: %.2f%%" % test_accuracy,"n_estimators:", ne, "max_depth:", md, "learning_rate:", lr)

Train accuracy: 96.67% Test accuracy: 86.67% n_estimators: 10 max_depth: 2 learning_rate: 0.001
Train accuracy: 96.67% Test accuracy: 86.67% n_estimators: 10 max_depth: 2 learning_rate: 0.01
Train accuracy: 98.89% Test accuracy: 90.00% n_estimators: 10 max_depth: 2 learning_rate: 0.1
Train accuracy: 98.89% Test accuracy: 90.00% n_estimators: 10 max_depth: 2 learning_rate: 0.2
Train accuracy: 98.89% Test accuracy: 90.00% n_estimators: 10 max_depth: 2 learning_rate: 0.3
Train accuracy: 98.89% Test accuracy: 86.67% n_estimators: 10 max_depth: 3 learning_rate: 0.001
Train accuracy: 98.89% Test accuracy: 86.67% n_estimators: 10 max_depth: 3 learning_rate: 0.01
Train accuracy: 100.00% Test accuracy: 90.00% n_estimators: 10 max_depth: 3 learning_rate: 0.1
Train accuracy: 100.00% Test accuracy: 90.00% n_estimators: 10 max_depth: 3 learning_rate: 0.2
Train accuracy: 100.00% Test accuracy: 90.00% n_estimators: 10 max_depth: 3 learning_rate: 0.3
Train accuracy: 98.89% Test accuracy: 86.67% n_esti

A la vista de los resultados cogeremos los siguientes parámetros:

In [91]:
N_ESTIMATOR = 200
MAX_DEPTH = 2
LEARNING_RATE = 0.01

Veamos el árbol con los nuevos parámetros.

In [92]:
model = XGBClassifier(objective='binary:logistic', n_estimators=N_ESTIMATOR, max_depth=MAX_DEPTH, learning_rate=LEARNING_RATE, seed=RANDOM_STATE)

model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

train_accuracy = model.score(X_train, y_train) *100
test_accuracy = model.score(X_test, y_test) *100

print("Train accuracy: %.2f%%" % train_accuracy, "Test accuracy: %.2f%%" % test_accuracy)

Train accuracy: 98.89% Test accuracy: 90.00%


Hemos obtenido una precisión del 98% en el conjunto de datos de entrenamiento y un 90% en el conjunto de datos de test. Lo cual nos muestra una alta tasa de acierto sin obtener sobreaprendizaje en el conjunto de datos de entrenamiento ya que se generaliza de manera adecuada para nuevos datos.

Hemos visto que la profundidad máxima del árbol es el parametro mas importante, ya que si es muy grande se produce sobreaprendizaje y si es muy pequeña no se generaliza bien.

## Conclusiones

Hemos visto que el XGBoost es un buen algoritmo para la construcción de árboles de decisión, ya que permite obtener una tasa de acierto muy alta sin sobreaprendizaje. Además, nos obtenemos los parámetros más importantes para el problema que de manera sencilla. 